In [66]:
# from google.colab import drive
# drive.mount('/content/drive')

In [67]:
# cd /content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification

# Imports

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import calendar
import warnings
from math import sin, cos, sqrt, atan2, radians,asin

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
# import lightgbm as lgb
# import xgboost as xgb
from sklearn.metrics import roc_auc_score
# from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
warnings.filterwarnings('ignore')
# from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score,f1_score


# Load Data

In [69]:
# train=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Train_FeatureEngineering_4_without_droping_any_feature.csv")
# test=pd.read_csv("/content/drive/My Drive/University/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/features/Test_FeatureEngineering_4.csv")
# for other account
train=pd.read_csv("features/Train_FeatureEngineering_6_fill_null_with_0_changed_dtype.csv")
test=pd.read_csv("features/Test_FeatureEngineering_6_fill_null_with_0_changed_dtype.csv")

print("Shape of Training Data",train.shape)
print("Shape of Testing Data", test.shape)


Shape of Training Data (17060, 25)
Shape of Testing Data (8576, 24)


In [70]:
train.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'label', 'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

In [71]:
test.columns

Index(['tripid', 'additional_fare', 'duration', 'meter_waiting',
       'meter_waiting_fare', 'meter_waiting_till_pickup', 'pickup_time',
       'drop_time', 'pick_lat', 'pick_lon', 'drop_lat', 'drop_lon', 'fare',
       'additional_fare_is_outlier', 'meter_waiting_fare_is_outlier',
       'fare_is_outlier', 'pickup_date', 'pickup_day', 'pickup_hour',
       'pickup_day_of_week', 'pickup_month', 'pickup_year', 'is_outlier_loc',
       'trip_distance'],
      dtype='object')

# Data_Preprocessing

In [72]:
# drop_columns_train=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3']
# drop_columns_test=['tripid','pickup_time','drop_time','pick_lat','pick_lon','drop_lat','drop_lon','pickup_date','pickup_latitude_round3','pickup_longitude_round3','dropoff_latitude_round3','dropoff_longitude_round3','is_outlier_loc']

drop_columns_train=['tripid','pickup_time','drop_time','pickup_date']
drop_columns_test=['tripid','pickup_time','drop_time','pickup_date']

In [73]:
train_1=train.drop(drop_columns_train,axis=1)
test_1=test.drop(drop_columns_test,axis=1)
print("Shape of Training Data after dropping columns",train_1.shape)
print("Shape of Testing Data after dropping columns",test_1.shape)

Shape of Training Data after dropping columns (17060, 21)
Shape of Testing Data after dropping columns (8576, 20)


In [74]:
# def encodeDays(day_of_week):
#   day_dict={'Sunday':0,'Monday':1,'Tuesday':2,'Wednesday':3,'Thursday':4,'Friday':5,'Saturday':6}
#   return day_dict[day_of_week]
# def encodeLabel(label):
#   label_dict={'correct':1,'incorrect':0}
#   return label_dict[label]
# def clean_data(data,isTrain):
#   data['pickup_day_of_week']=data['pickup_day_of_week'].apply(lambda x:encodeDays(x))
#   if(isTrain):
#     data['label']=data['label'].apply(lambda x:encodeLabel(x))
#   return data

# train_1=clean_data(train_1,True)
# test_1=clean_data(test_1,False)
# print("Shape of Training Data after cleaning ",train_1.shape)
# print("Shape of Testing Data after cleaning", test_1.shape)

In [75]:
test_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8576 entries, 0 to 8575
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                8576 non-null   float64
 1   duration                       8576 non-null   int64  
 2   meter_waiting                  8576 non-null   int64  
 3   meter_waiting_fare             8576 non-null   float64
 4   meter_waiting_till_pickup      8576 non-null   int64  
 5   pick_lat                       8576 non-null   float64
 6   pick_lon                       8576 non-null   float64
 7   drop_lat                       8576 non-null   float64
 8   drop_lon                       8576 non-null   float64
 9   fare                           8576 non-null   float64
 10  additional_fare_is_outlier     8576 non-null   int64  
 11  meter_waiting_fare_is_outlier  8576 non-null   int64  
 12  fare_is_outlier                8576 non-null   i

In [76]:
columns_titles = ['additional_fare','duration','meter_waiting','meter_waiting_fare','meter_waiting_till_pickup','pick_lat','pick_lon','drop_lat','drop_lon','fare','additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_day','pickup_hour','pickup_day_of_week','pickup_month','pickup_year','is_outlier_loc','trip_distance','label']
train_1=train_1.reindex(columns=columns_titles)
train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17060 entries, 0 to 17059
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   additional_fare                17060 non-null  float64
 1   duration                       17060 non-null  float64
 2   meter_waiting                  17060 non-null  float64
 3   meter_waiting_fare             17060 non-null  float64
 4   meter_waiting_till_pickup      17060 non-null  float64
 5   pick_lat                       17060 non-null  float64
 6   pick_lon                       17060 non-null  float64
 7   drop_lat                       17060 non-null  float64
 8   drop_lon                       17060 non-null  float64
 9   fare                           17060 non-null  float64
 10  additional_fare_is_outlier     17060 non-null  int64  
 11  meter_waiting_fare_is_outlier  17060 non-null  int64  
 12  fare_is_outlier                17060 non-null 

In [77]:
categorical_var = ['additional_fare_is_outlier','meter_waiting_fare_is_outlier','fare_is_outlier','pickup_day','pickup_hour','pickup_day_of_week','pickup_month','pickup_year','is_outlier_loc'] 
categorical_var_indices = [train_1.columns.get_loc(c) for c in categorical_var if c in train_1]
print('\nCategorical Variables indices : ',categorical_var_indices)


Categorical Variables indices :  [10, 11, 12, 13, 14, 15, 16, 17, 18]


In [78]:
# train_1=pd.get_dummies(train_1)
# test_1=pd.get_dummies(test_1)

# print("Shape of Training Data after One Hot Encoding",train_1.shape)
# print("Shape of Testing Data after One Hot Encoding",test_1.shape)

In [79]:
# print('Old size: %d' % len(train))
# train_1 = train_1.dropna()
# print('New size: %d' % len(train))

In [80]:
X=train_1.drop(['label'],axis=1)
y=train_1['label']

In [81]:
#split data into train and validation data
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)
print("Number of records in training data ",X_train.shape[0])
print("Number of records in validation data ",X_test.shape[0])

Number of records in training data  13648
Number of records in validation data  3412


# Classifiers

## XGBoost CLassifier

In [82]:
# model = XGBClassifier()
# model.fit(X_train, y_train)

## Linear Regression Classifier

In [83]:
# model = LinearRegression()
# model.fit(X_train,y_train)
# # y_pred=model.predict(X_test)
# # lm_rmse=np.sqrt(mean_squared_error(y_pred, y_test))
# # print("RMSE for Linear Regression is ",lm_rmse)


## LGBM Classifier

In [84]:
# estimator = lgb.LGBMClassifier(learning_rate = 0.125, metric = 'l1', 
#                         n_estimators = 20, num_leaves = 38)


# param_grid = {
#     'n_estimators': [x for x in range(24,40,2)],
#     'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
# gridsearch = GridSearchCV(estimator, param_grid)

# gridsearch.fit(X_train, y_train,
#         eval_set = [(X_test, y_test)],
#         eval_metric = ['auc', 'binary_logloss'],
#         early_stopping_rounds = 5)

In [85]:
# gbm = lgb.LGBMClassifier(boosting_type='gbdt', class_weight=None,
#                                       colsample_bytree=1.0,
#                                       importance_type='split',
#                                       learning_rate=0.125, max_depth=-1,
#                                       metric='l1', min_child_samples=20,
#                                       min_child_weight=0.001,
#                                       min_split_gain=0.0, n_estimators=20,
#                                       n_jobs=-1, num_leaves=38, objective=None,
#                                       random_state=None, reg_alpha=0.0,
#                                       reg_lambda=0.0, silent=True,
#                                       subsample=1.0, subsample_for_bin=200000,
#                                       subsample_freq=0)


# gbm.fit(X, y,
#         eval_set=[(X_test, y_test)],
#         eval_metric=['auc', 'binary_logloss'],
# early_stopping_rounds=5)

In [86]:
# ax = lgb.plot_importance(gbm, height = 0.4, 
#                          max_num_features = 50, 
#                          xlim = (0,100), ylim = (0,23), 
#                          figsize = (10,6))
# plt.show()

In [87]:
# sorted(gbm.feature_importances_,reverse=True)

In [88]:

# # The code below aims to drop  to keep the features that are included in the most important features. 
# temp = 0 
# total = sum(gbm.feature_importances_)
# for feature in sorted(gbm.feature_importances_, reverse=True):
#     temp+=feature
#     print(feature)
#     if temp/total >= 0.85:
#         print(feature,temp/total) # stop when we 
#         break

In [89]:
# #The above means let go of all variables after PAY_AMT_5
# y_pred_prob = gbm.predict_proba(test_1)[:, 1]
# # auc_roc_0 = str(roc_auc_score(y_test, y_pred_prob)) # store AUC score without dimensionality reduction
# # print('AUC without dimensionality reduction: \n' + auc_roc_0)

In [90]:
X

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,additional_fare_is_outlier,meter_waiting_fare_is_outlier,fare_is_outlier,pickup_day,pickup_hour,pickup_day_of_week,pickup_month,pickup_year,is_outlier_loc,trip_distance
0,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,0,0,0,1,0,5,11,2019,0,3.164501
1,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,0,0,0,1,0,5,11,2019,0,1.968540
2,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,0,0,0,1,1,5,11,2019,0,3.917991
3,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,0,0,0,1,2,5,11,2019,0,0.535588
4,0.0,0.0,0.0,0.0000,0.0,6.87441,79.8615,6.84478,79.9290,358.39,0,0,0,1,3,5,11,2019,0,5.062797
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17055,10.5,838.0,93.0,5.4219,451.0,7.29073,80.6367,7.28891,80.6557,198.26,0,0,0,31,22,5,1,2020,0,1.308220
17056,10.5,2151.0,428.0,0.0000,39.0,6.90569,79.8516,6.95089,79.9389,581.23,0,0,0,31,23,5,1,2020,0,6.753297
17057,10.5,263.0,9.0,0.0000,110.0,7.09210,79.9000,7.10135,79.9017,76.20,0,0,0,31,23,5,1,2020,0,0.649655
17058,10.5,858.0,115.0,0.0000,317.0,6.94540,79.8768,6.93574,79.9010,133.31,0,0,0,31,23,5,1,2020,0,1.788975


In [91]:
# #We can choose to drop the last 6 features from in our new model to reduce dimensionality, and thus save training time and space

# X = X_train.drop(['additional_fare','pickup_day_of_week','pickup_month','pickup_day'], axis=1)

## CatBoost

In [92]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

model = CatBoostClassifier(iterations=100000, 
                           cat_features=categorical_var_indices, 
                           task_type="GPU",
                           devices='0:1')
model.fit(X,
          y,
          verbose=True)




Learning rate set to 0.000466
0:	learn: 0.6925757	total: 86.1ms	remaining: 2h 23m 28s
1:	learn: 0.6920052	total: 170ms	remaining: 2h 21m 54s
2:	learn: 0.6915179	total: 249ms	remaining: 2h 18m 16s
3:	learn: 0.6909596	total: 328ms	remaining: 2h 16m 44s
4:	learn: 0.6904579	total: 406ms	remaining: 2h 15m 24s
5:	learn: 0.6899295	total: 481ms	remaining: 2h 13m 34s
6:	learn: 0.6894032	total: 551ms	remaining: 2h 11m 5s
7:	learn: 0.6888298	total: 608ms	remaining: 2h 6m 38s
8:	learn: 0.6882909	total: 679ms	remaining: 2h 5m 48s
9:	learn: 0.6878127	total: 745ms	remaining: 2h 4m 13s
10:	learn: 0.6873443	total: 812ms	remaining: 2h 2m 58s
11:	learn: 0.6867673	total: 876ms	remaining: 2h 1m 36s
12:	learn: 0.6862108	total: 937ms	remaining: 2h 10s
13:	learn: 0.6857314	total: 1.01s	remaining: 1h 59m 47s
14:	learn: 0.6851789	total: 1.07s	remaining: 1h 59m 4s
15:	learn: 0.6847060	total: 1.14s	remaining: 1h 58m 16s
16:	learn: 0.6842143	total: 1.19s	remaining: 1h 57m 5s
17:	learn: 0.6837381	total: 1.26s	remai

150:	learn: 0.6209670	total: 9.47s	remaining: 1h 44m 23s
151:	learn: 0.6205531	total: 9.53s	remaining: 1h 44m 23s
152:	learn: 0.6200661	total: 9.59s	remaining: 1h 44m 15s
153:	learn: 0.6195942	total: 9.65s	remaining: 1h 44m 14s
154:	learn: 0.6191903	total: 9.71s	remaining: 1h 44m 13s
155:	learn: 0.6187964	total: 9.76s	remaining: 1h 44m 8s
156:	learn: 0.6183941	total: 9.82s	remaining: 1h 44m 7s
157:	learn: 0.6179994	total: 9.89s	remaining: 1h 44m 9s
158:	learn: 0.6176183	total: 9.96s	remaining: 1h 44m 11s
159:	learn: 0.6171778	total: 10s	remaining: 1h 44m 10s
160:	learn: 0.6166661	total: 10.1s	remaining: 1h 44m 8s
161:	learn: 0.6161593	total: 10.1s	remaining: 1h 44m 6s
162:	learn: 0.6157700	total: 10.2s	remaining: 1h 44m 6s
163:	learn: 0.6153639	total: 10.3s	remaining: 1h 44m 4s
164:	learn: 0.6149768	total: 10.3s	remaining: 1h 44m 3s
165:	learn: 0.6145819	total: 10.4s	remaining: 1h 44m 4s
166:	learn: 0.6142035	total: 10.4s	remaining: 1h 44m 3s
167:	learn: 0.6138007	total: 10.5s	remainin

298:	learn: 0.5618509	total: 18.4s	remaining: 1h 42m 19s
299:	learn: 0.5614389	total: 18.5s	remaining: 1h 42m 19s
300:	learn: 0.5610090	total: 18.5s	remaining: 1h 42m 19s
301:	learn: 0.5606822	total: 18.6s	remaining: 1h 42m 20s
302:	learn: 0.5603490	total: 18.7s	remaining: 1h 42m 20s
303:	learn: 0.5599878	total: 18.7s	remaining: 1h 42m 20s
304:	learn: 0.5595412	total: 18.8s	remaining: 1h 42m 19s
305:	learn: 0.5592232	total: 18.8s	remaining: 1h 42m 10s
306:	learn: 0.5588462	total: 18.9s	remaining: 1h 42m 10s
307:	learn: 0.5584445	total: 18.9s	remaining: 1h 42m 9s
308:	learn: 0.5581187	total: 19s	remaining: 1h 42m 2s
309:	learn: 0.5577810	total: 19s	remaining: 1h 42m 2s
310:	learn: 0.5573864	total: 19.1s	remaining: 1h 42m 2s
311:	learn: 0.5569890	total: 19.2s	remaining: 1h 42m 1s
312:	learn: 0.5566338	total: 19.2s	remaining: 1h 42m 1s
313:	learn: 0.5561950	total: 19.3s	remaining: 1h 42m 1s
314:	learn: 0.5558713	total: 19.3s	remaining: 1h 42m 1s
315:	learn: 0.5554864	total: 19.4s	remainin

446:	learn: 0.5111536	total: 27.5s	remaining: 1h 42m 4s
447:	learn: 0.5108577	total: 27.6s	remaining: 1h 42m 4s
448:	learn: 0.5105841	total: 27.6s	remaining: 1h 42m 3s
449:	learn: 0.5102162	total: 27.7s	remaining: 1h 42m 3s
450:	learn: 0.5098368	total: 27.7s	remaining: 1h 42m 3s
451:	learn: 0.5094796	total: 27.8s	remaining: 1h 42m 4s
452:	learn: 0.5091131	total: 27.9s	remaining: 1h 42m 4s
453:	learn: 0.5088315	total: 27.9s	remaining: 1h 42m 5s
454:	learn: 0.5085503	total: 28s	remaining: 1h 42m 5s
455:	learn: 0.5082564	total: 28.1s	remaining: 1h 42m 5s
456:	learn: 0.5079242	total: 28.1s	remaining: 1h 42m 6s
457:	learn: 0.5076554	total: 28.2s	remaining: 1h 42m 7s
458:	learn: 0.5073723	total: 28.3s	remaining: 1h 42m 7s
459:	learn: 0.5070713	total: 28.3s	remaining: 1h 42m 6s
460:	learn: 0.5067224	total: 28.4s	remaining: 1h 42m 6s
461:	learn: 0.5064472	total: 28.4s	remaining: 1h 42m 5s
462:	learn: 0.5061073	total: 28.5s	remaining: 1h 42m 5s
463:	learn: 0.5057956	total: 28.6s	remaining: 1h 4

594:	learn: 0.4689436	total: 36.6s	remaining: 1h 41m 57s
595:	learn: 0.4686326	total: 36.7s	remaining: 1h 41m 57s
596:	learn: 0.4683517	total: 36.7s	remaining: 1h 41m 57s
597:	learn: 0.4681012	total: 36.8s	remaining: 1h 41m 57s
598:	learn: 0.4678720	total: 36.9s	remaining: 1h 41m 56s
599:	learn: 0.4676483	total: 36.9s	remaining: 1h 41m 57s
600:	learn: 0.4674248	total: 37s	remaining: 1h 41m 57s
601:	learn: 0.4671973	total: 37s	remaining: 1h 41m 57s
602:	learn: 0.4669682	total: 37.1s	remaining: 1h 41m 57s
603:	learn: 0.4666973	total: 37.2s	remaining: 1h 41m 57s
604:	learn: 0.4664067	total: 37.2s	remaining: 1h 41m 56s
605:	learn: 0.4661505	total: 37.3s	remaining: 1h 41m 56s
606:	learn: 0.4659237	total: 37.4s	remaining: 1h 41m 56s
607:	learn: 0.4656316	total: 37.4s	remaining: 1h 41m 56s
608:	learn: 0.4653923	total: 37.5s	remaining: 1h 41m 56s
609:	learn: 0.4651733	total: 37.5s	remaining: 1h 41m 56s
610:	learn: 0.4649294	total: 37.6s	remaining: 1h 41m 56s
611:	learn: 0.4646561	total: 37.7s	

...

7129:	learn: 0.1758941	total: 7m 22s	remaining: 1h 36m 1s
7130:	learn: 0.1758925	total: 7m 22s	remaining: 1h 36m 1s
7131:	learn: 0.1758899	total: 7m 22s	remaining: 1h 36m 1s
7132:	learn: 0.1758778	total: 7m 22s	remaining: 1h 36m 1s
7133:	learn: 0.1758696	total: 7m 22s	remaining: 1h 36m 1s
7134:	learn: 0.1758628	total: 7m 22s	remaining: 1h 36m 1s
7135:	learn: 0.1758588	total: 7m 22s	remaining: 1h 36m 1s
7136:	learn: 0.1758533	total: 7m 22s	remaining: 1h 36m
7137:	learn: 0.1758506	total: 7m 22s	remaining: 1h 36m
7138:	learn: 0.1758431	total: 7m 22s	remaining: 1h 36m
7139:	learn: 0.1758410	total: 7m 22s	remaining: 1h 36m
7140:	learn: 0.1758400	total: 7m 23s	remaining: 1h 36m
7141:	learn: 0.1758364	total: 7m 23s	remaining: 1h 36m
7142:	learn: 0.1758342	total: 7m 23s	remaining: 1h 36m
7143:	learn: 0.1758243	total: 7m 23s	remaining: 1h 36m
7144:	learn: 0.1758220	total: 7m 23s	remaining: 1h 36m
7145:	learn: 0.1758172	total: 7m 23s	remaining: 1h 36m
7146:	learn: 0.1758096	total: 7m 23s	remaini

7273:	learn: 0.1751467	total: 7m 31s	remaining: 1h 35m 52s
7274:	learn: 0.1751432	total: 7m 31s	remaining: 1h 35m 52s
7275:	learn: 0.1751420	total: 7m 31s	remaining: 1h 35m 51s
7276:	learn: 0.1751314	total: 7m 31s	remaining: 1h 35m 51s
7277:	learn: 0.1751306	total: 7m 31s	remaining: 1h 35m 51s
7278:	learn: 0.1751281	total: 7m 31s	remaining: 1h 35m 51s
7279:	learn: 0.1751198	total: 7m 31s	remaining: 1h 35m 51s
7280:	learn: 0.1751101	total: 7m 31s	remaining: 1h 35m 51s
7281:	learn: 0.1751046	total: 7m 31s	remaining: 1h 35m 51s
7282:	learn: 0.1751013	total: 7m 31s	remaining: 1h 35m 51s
7283:	learn: 0.1750960	total: 7m 31s	remaining: 1h 35m 51s
7284:	learn: 0.1750836	total: 7m 31s	remaining: 1h 35m 51s
7285:	learn: 0.1750776	total: 7m 31s	remaining: 1h 35m 51s
7286:	learn: 0.1750762	total: 7m 32s	remaining: 1h 35m 51s
7287:	learn: 0.1750711	total: 7m 32s	remaining: 1h 35m 51s
7288:	learn: 0.1750630	total: 7m 32s	remaining: 1h 35m 51s
7289:	learn: 0.1750562	total: 7m 32s	remaining: 1h 35m 5

7413:	learn: 0.1743831	total: 7m 39s	remaining: 1h 35m 43s
7414:	learn: 0.1743811	total: 7m 39s	remaining: 1h 35m 43s
7415:	learn: 0.1743719	total: 7m 40s	remaining: 1h 35m 42s
7416:	learn: 0.1743703	total: 7m 40s	remaining: 1h 35m 42s
7417:	learn: 0.1743669	total: 7m 40s	remaining: 1h 35m 42s
7418:	learn: 0.1743620	total: 7m 40s	remaining: 1h 35m 42s
7419:	learn: 0.1743499	total: 7m 40s	remaining: 1h 35m 42s
7420:	learn: 0.1743387	total: 7m 40s	remaining: 1h 35m 42s
7421:	learn: 0.1743356	total: 7m 40s	remaining: 1h 35m 42s
7422:	learn: 0.1743327	total: 7m 40s	remaining: 1h 35m 42s
7423:	learn: 0.1743306	total: 7m 40s	remaining: 1h 35m 42s
7424:	learn: 0.1743295	total: 7m 40s	remaining: 1h 35m 42s
7425:	learn: 0.1743287	total: 7m 40s	remaining: 1h 35m 42s
7426:	learn: 0.1743243	total: 7m 40s	remaining: 1h 35m 42s
7427:	learn: 0.1743120	total: 7m 40s	remaining: 1h 35m 42s
7428:	learn: 0.1743048	total: 7m 40s	remaining: 1h 35m 42s
7429:	learn: 0.1742949	total: 7m 40s	remaining: 1h 35m 4

7553:	learn: 0.1736999	total: 7m 48s	remaining: 1h 35m 34s
7554:	learn: 0.1736944	total: 7m 48s	remaining: 1h 35m 34s
7555:	learn: 0.1736881	total: 7m 48s	remaining: 1h 35m 34s
7556:	learn: 0.1736814	total: 7m 48s	remaining: 1h 35m 34s
7557:	learn: 0.1736730	total: 7m 48s	remaining: 1h 35m 34s
7558:	learn: 0.1736679	total: 7m 48s	remaining: 1h 35m 34s
7559:	learn: 0.1736649	total: 7m 48s	remaining: 1h 35m 34s
7560:	learn: 0.1736621	total: 7m 49s	remaining: 1h 35m 34s
7561:	learn: 0.1736545	total: 7m 49s	remaining: 1h 35m 34s
7562:	learn: 0.1736472	total: 7m 49s	remaining: 1h 35m 34s
7563:	learn: 0.1736401	total: 7m 49s	remaining: 1h 35m 34s
7564:	learn: 0.1736359	total: 7m 49s	remaining: 1h 35m 34s
7565:	learn: 0.1736272	total: 7m 49s	remaining: 1h 35m 33s
7566:	learn: 0.1736154	total: 7m 49s	remaining: 1h 35m 33s
7567:	learn: 0.1736085	total: 7m 49s	remaining: 1h 35m 33s
7568:	learn: 0.1736014	total: 7m 49s	remaining: 1h 35m 33s
7569:	learn: 0.1735973	total: 7m 49s	remaining: 1h 35m 3

7693:	learn: 0.1729915	total: 7m 57s	remaining: 1h 35m 25s
7694:	learn: 0.1729829	total: 7m 57s	remaining: 1h 35m 25s
7695:	learn: 0.1729786	total: 7m 57s	remaining: 1h 35m 25s
7696:	learn: 0.1729725	total: 7m 57s	remaining: 1h 35m 25s
7697:	learn: 0.1729701	total: 7m 57s	remaining: 1h 35m 25s
7698:	learn: 0.1729690	total: 7m 57s	remaining: 1h 35m 25s
7699:	learn: 0.1729665	total: 7m 57s	remaining: 1h 35m 25s
7700:	learn: 0.1729566	total: 7m 57s	remaining: 1h 35m 25s
7701:	learn: 0.1729495	total: 7m 57s	remaining: 1h 35m 24s
7702:	learn: 0.1729463	total: 7m 57s	remaining: 1h 35m 24s
7703:	learn: 0.1729365	total: 7m 57s	remaining: 1h 35m 24s
7704:	learn: 0.1729342	total: 7m 57s	remaining: 1h 35m 24s
7705:	learn: 0.1729314	total: 7m 57s	remaining: 1h 35m 24s
7706:	learn: 0.1729275	total: 7m 58s	remaining: 1h 35m 24s
7707:	learn: 0.1729246	total: 7m 58s	remaining: 1h 35m 24s
7708:	learn: 0.1729207	total: 7m 58s	remaining: 1h 35m 24s
7709:	learn: 0.1729181	total: 7m 58s	remaining: 1h 35m 2

#Test Model

In [ ]:
predictions = model.predict(X_test)
predictions = [round(value) for value in predictions]
np.unique(predictions , return_counts=True) 

In [101]:
accuracy = accuracy_score(y_test, predictions)
f1_macro = f1_score(y_test, predictions, average='macro')
f1_binary = f1_score(y_test, predictions, average='binary')
print("Accuracy: %.2f%%" % (accuracy * 100.0))
print("Macro F1: %.2f%%" % (f1_macro * 100.0))
print("Binary F1: %.2f%%" % (f1_binary * 100.0))

Accuracy: 96.16%
Macro F1: 86.48%
Binary F1: 97.92%


## Results

### catboost with categorical features 100000 epochs 
Accuracy: 96.16%
Macro F1: 86.48%
Binary F1: 97.92%
Submission_score : 0.97375

### catboost without dropping any feature with 400000 epochs

Accuracy: 97.41%
Macro F1: 90.87%
Binary F1: 98.60%
Submission Score : 0.97817

### catboost without dropping any feature  with 100000 epochs

Accuracy: 97.35%
Macro F1: 90.63%
Binary F1: 98.56%
Submission Score : 0.97848

### catboost after normalizing to z score with 10000 epochs

Accuracy: 96.70%
Macro F1: 87.82%
Binary F1: 98.22%

### catboost without dropping any feature with 10000 epochs

Accuracy: 96.82%
Macro F1: 88.95%
Binary F1: 98.27%
Submission Score : 0.97788

# Save Predictions

In [ ]:
predictions_1 = model.predict(test_1)
# predictions_1 = [round(value) for value in y_pred_1]
np.unique(predictions_1 , return_counts=True)

In [ ]:
submission_df = pd.read_csv("predications_submission.csv", 
                            index_col="tripid")
submission_df['prediction'] = predictions_1

submission_df.to_csv('predictions/catboost_categorical_100000_epochs.csv')

In [ ]:
#  y_pred = pd.DataFrame({'prediction':predictions_1})
#  y_pred["prediction"] = y_pred["prediction"].astype(int)
#  y_pred['prediction'] = y_pred['prediction'].apply(lambda x: 0 if x < 0.5 else 1)
#  y_pred.to_csv("predictions/predications_catboost_6_features_3_100000epochs.csv")
# y_pred.to_csv("/content/drive/My Drive/UNI/SEM7/CS4622_Machine_Learning/Project_1/Fair_Classification/predictions/predications_catboost_6_features_3_400000epochs.csv")